<a href="https://colab.research.google.com/github/ancestor9/2025_Fall_AI-Model-Operations-MLOps/blob/main/week07/database_python/sqlalchemy_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. sqlalchemy 소개**

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker

# 1. 데이터베이스 연결 설정 및 생성 (Engine) : 도서관 건물 만들기
- 'sqlite:///test.db'는 현재 디렉토리에 'test.db' 파일을 생성하거나 연결
- echo=True는 SQLAlchemy가 실행하는 모든 SQL 쿼리를 터미널에 출력

In [ ]:
engine = create_engine("sqlite:///test.db", echo=True)


# 2. 모델(테이블 스키마) 정의를 위한 베이스 클래스 생성 : 책장 Template 만들기

In [ ]:
Base = declarative_base()

# 3. 테이블 모델 정의 (Table Schema) : 실제 책장 만들기

In [ ]:
class User(Base):
    # 데이터베이스 테이블 이름 설정
    __tablename__ = 'users'

    # 컬럼 정의: primary_key=True는 기본 키를 의미하며, autoincrement=True는 자동으로 증가
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)

    # 객체를 출력할 때 보기 쉽게 표현하는 메서드
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

# 4. 데이터베이스 테이블 생성 (DDL) :
- Base에 정의된 모든 모델(테이블)을 데이터베이스에 생성

In [ ]:
Base.metadata.create_all(engine)


2025-10-20 09:26:10,723 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-10-20 09:26:10,725 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("users")


2025-10-20 09:26:10,726 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2025-10-20 09:26:10,728 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


# 5. 세션 생성 (Session) : DB 접속 입장 티켓 발급
- sessionmaker는 세션을 생성할 때 필요한 설정(예: 어떤 데이터베이스 엔진(engine)을 사용할지, 트랜잭션 옵션은 무엇인지 등)을 미리 정의하고 저장

In [ ]:
Session = sessionmaker(bind=engine)

#### 데이터베이스와의 모든 상호작용은 세션 객체(Object)를 통해 이루어짐
#### 세션 팩토리(Session)를 호출하여 **실제 데이터베이스 연결을 위한 세션 객체(session)**를 생성
#### 이 session 객체가 바로 데이터베이스에 쿼리를 보내고, 데이터를 추가/수정/삭제하며, 트랜잭션을 관리하는 'DB 접속 입장 티켓' 역할

In [ ]:
session = Session()

# -------------------------------------------------------------
# CRUD (Create, Read, Update, Delete) 작업 시작
# -------------------------------------------------------------


In [ ]:
# C (Create) - 데이터 생성 및 삽입
# -----------------------------------------
print("\n--- [C]reate: 새 사용자 추가 ---")
# User 객체(행) 생성
user1 = User(name='spongebob', fullname='Spongebob Squarepants')
user2 = User(name='patrick', fullname='Patrick Star')

# 세션에 객체 추가 (이 시점에서는 데이터베이스에 반영되지 않음)
session.add(user1)
session.add(user2)

# 세션의 변경 사항을 데이터베이스에 커밋(반영)
session.commit()
print(f"추가된 사용자 1: {user1}")
print(f"추가된 사용자 2: {user2}")

# R (Read) - 데이터 읽기 (조회)
# -----------------------------------------
print("\n--- [R]ead: 모든 사용자 조회 ---")
# User 테이블의 모든 행(객체)을 조회
for user in session.query(User).all():
    print(user)

print("\n--- [R]ead: 특정 사용자 (ID 1) 조회 ---")
# 기본 키를 사용하여 특정 행(객체) 조회
user_to_read = session.query(User).get(1)
if user_to_read:
    print(user_to_read)

# U (Update) - 데이터 수정
# -----------------------------------------
print("\n--- [U]pdate: 사용자 이름 수정 ---")
# 조회된 객체의 속성을 수정
if user_to_read:
    user_to_read.fullname = 'S. Squarepants'
    print(f"수정 후 (커밋 전): {user_to_read}")

    # 변경 사항 커밋
    session.commit()
    print("수정 사항 커밋 완료.")

# D (Delete) - 데이터 삭제
# -----------------------------------------
print("\n--- [D]elete: 사용자 (ID 2) 삭제 ---")
# ID가 2인 사용자 조회
user_to_delete = session.query(User).filter_by(name='patrick').first()

if user_to_delete:
    # 세션에서 객체 삭제
    session.delete(user_to_delete)
    print(f"삭제할 사용자: {user_to_delete}")

    # 변경 사항 커밋
    session.commit()
    print("삭제 사항 커밋 완료.")

# R (Read) - 최종 확인
print("\n--- [R]ead: 삭제 후 남은 사용자 확인 ---")
for user in session.query(User).all():
    print(user)



--- [C]reate: 새 사용자 추가 ---
2025-10-20 09:26:12,722 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-10-20 09:26:12,725 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname) VALUES (?, ?) RETURNING id


INFO:sqlalchemy.engine.Engine:INSERT INTO users (name, fullname) VALUES (?, ?) RETURNING id


2025-10-20 09:26:12,733 INFO sqlalchemy.engine.Engine [generated in 0.00017s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('spongebob', 'Spongebob Squarepants')


...

2025-10-20 09:26:12,879 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users


INFO:sqlalchemy.engine.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users


2025-10-20 09:26:12,883 INFO sqlalchemy.engine.Engine [cached since 0.08642s ago] ()


INFO:sqlalchemy.engine.Engine:[cached since 0.08642s ago] ()


User(id=1, name='spongebob', fullname='S. Squarepants')
User(id=2, name='spongebob', fullname='Spongebob Squarepants')
User(id=3, name='spongebob', fullname='Spongebob Squarepants')
User(id=4, name='spongebob', fullname='Spongebob Squarepants')
User(id=5, name='spongebob', fullname='Spongebob Squarepants')


# 세션 닫기

In [ ]:
session.close()
# test.db 파일이 생성되었으며, 실행 후에도 유지

2025-10-20 09:26:14,487 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


# **2. DB 테이블을 데이터프레임으로**
- with statement 구문 사용

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

# Database connection URL (assuming test.db is in the /content directory)
db_url = "sqlite:////content/test.db"
engine = create_engine(db_url)

# SQL query to select all data from the "users" table
query = "SELECT * FROM users"


cursor = engine.connect()

df = pd.read_sql(text(query), cursor)
df

,id,name,fullname
0,1,spongebob,S. Squarepants
1,2,spongebob,Spongebob Squarepants
2,3,spongebob,Spongebob Squarepants
3,4,spongebob,Spongebob Squarepants


In [ ]:
# Read the data into a pandas DataFrame
try:
    with engine.connect() as connection:
        df = pd.read_sql(text(query), connection)

    # Display the DataFrame
    display(df)

except Exception as e:
    print(f"An error occurred: {e}")

,id,name,fullname
0,1,spongebob,S. Squarepants
1,2,spongebob,Spongebob Squarepants
2,3,spongebob,Spongebob Squarepants
3,4,spongebob,Spongebob Squarepants


# **3. SessionLocal()**

- 전역 세션을 사용하는 방식에서 벗어나 (모듈 레벨에서 session = Session()을 호출하여 단일 전역 세션을 생성하고 재사용)
- 함수 수준에서 독립적이고 격리된 세션을 생성, 사용, 종료하는 표준 패턴을 적용하여 안정성, 재사용성, 확장성을 높임

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker, Session
from typing import Generator

# -------------------------------------------------------------
# 1. 설정 및 기본 객체 정의 (모듈 레벨)
# -------------------------------------------------------------

# 데이터베이스 연결 설정
SQLALCHEMY_DATABASE_URL = "sqlite:///train.db"

# Engine 생성. SQLite는 기본적으로 check_same_thread=False가 필요할 수 있습니다.
# (멀티 스레드 환경에서 세션을 공유하지 않아도 필요할 수 있음)
engine = create_engine(
    SQLALCHEMY_DATABASE_URL,
    echo=True, # 실행되는 SQL 쿼리를 출력
    connect_args={"check_same_thread": False}
)

# 모델 정의를 위한 베이스 클래스
Base = declarative_base()

# SessionLocal (세션 팩토리) 정의
# autocommit=False: 트랜잭션 수동 관리 (commit 필요)
# autoflush=False: flush를 명시적으로 호출할 때만 수행 (성능 향상)
# bind=engine: 생성될 세션이 사용할 Engine 지정
SessionLocal = sessionmaker(
    autocommit=False,
    autoflush=False,
    bind=engine
)

# -------------------------------------------------------------
# 2. 모델 정의
# -------------------------------------------------------------

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)

    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

# -------------------------------------------------------------
# 3. 유틸리티 함수 (데이터베이스 초기화 및 세션 관리)
# -------------------------------------------------------------

def initialize_db():
    """데이터베이스 테이블이 없으면 생성합니다."""
    print("--- [Initialization] 테이블 생성 ---")
    Base.metadata.create_all(bind=engine)

def get_db() -> Generator[Session, None, None]:
    """
    각 CRUD 작업에서 사용할 독립적인 세션을 생성하고,
    작업 완료 후 세션을 닫아주는 의존성 주입 유틸리티 함수.
    (with 문으로 사용하거나, yield를 통해 컨텍스트 관리자로 사용됨)
    """
    db = SessionLocal()
    try:
        # yield를 통해 세션을 반환하고, 함수 종료 시 finally 블록 실행을 보장
        yield db
    finally:
        db.close()
        print(">> Session Closed")


# -------------------------------------------------------------
# 4. CRUD 작업 함수 정의 (get_db()를 사용하여 독립적인 세션 사용)
# -------------------------------------------------------------

def create_users():
    """새로운 사용자를 생성하고 커밋합니다 (Create)."""
    # get_db()를 호출하여 새 세션을 얻고 with 문으로 자동으로 닫습니다.
    with next(get_db()) as db:
        print("\n--- [C]reate: 새 사용자 추가 ---")

        # 이전 실행 시 데이터가 남아있을 수 있으므로, 간단히 모두 삭제 후 시작 (테스트용)
        db.query(User).delete()
        db.commit()

        user1 = User(name='spongebob', fullname='Spongebob Squarepants')
        user2 = User(name='patrick', fullname='Patrick Star')

        db.add_all([user1, user2])
        db.commit() # 커밋

        # 커밋 후 ID를 받아오기 위해 객체 새로고침 (선택 사항)
        db.refresh(user1)
        db.refresh(user2)

        print(f"추가된 사용자 1: {user1}")
        print(f"추가된 사용자 2: {user2}")


def read_users():
    """모든 사용자를 조회합니다 (Read)."""
    # with next(get_db()) as db: 와 같은 표현이지만,
    # 간단한 조회는 단일 세션에서 처리하고 닫습니다.
    db = SessionLocal()
    print("\n--- [R]ead: 모든 사용자 조회 ---")
    users = db.query(User).all()
    for user in users:
        print(user)
    db.close()


def update_user(user_name: str, new_fullname: str):
    """특정 사용자의 전체 이름을 수정합니다 (Update)."""
    with next(get_db()) as db:
        print(f"\n--- [U]pdate: {user_name}의 이름 수정 ---")

        user_to_update = db.query(User).filter(User.name == user_name).first()

        if user_to_update:
            user_to_update.fullname = new_fullname
            print(f"수정 전: {user_to_update}")

            db.commit() # 커밋
            print("수정 사항 커밋 완료.")

            # 변경 사항 확인을 위해 새로고침
            db.refresh(user_to_update)
            print(f"수정 후: {user_to_update}")
        else:
            print(f"사용자 '{user_name}'를 찾을 수 없습니다.")


def delete_user(user_name: str):
    """특정 사용자를 삭제합니다 (Delete)."""
    with next(get_db()) as db:
        print(f"\n--- [D]elete: 사용자 {user_name} 삭제 ---")

        user_to_delete = db.query(User).filter(User.name == user_name).first()

        if user_to_delete:
            print(f"삭제할 사용자: {user_to_delete}")
            db.delete(user_to_delete)
            db.commit() # 커밋
            print("삭제 사항 커밋 완료.")
        else:
            print(f"사용자 '{user_name}'를 찾을 수 없습니다.")


# -------------------------------------------------------------
# 5. 메인 실행 로직
# -------------------------------------------------------------

if __name__ == "__main__":

    # 1. 데이터베이스 초기화
    initialize_db()

    # 2. C (Create) 실행
    create_users()

    # 3. R (Read) 실행
    read_users()

    # 4. U (Update) 실행
    update_user(user_name='spongebob', new_fullname='S. Squarepants')

    # 5. D (Delete) 실행
    delete_user(user_name='patrick')

    # 6. R (Read) - 최종 확인
    read_users()

# 모든 작업이 get_db() 유틸리티를 통해 독립적인 세션을 사용하고 명시적으로 닫습니다.

--- [Initialization] 테이블 생성 ---
2025-10-20 09:28:16,938 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-10-20 09:28:16,940 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("users")


2025-10-20 09:28:16,943 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname 
FROM users


2025-10-20 09:28:17,120 INFO sqlalchemy.engine.Engine [cached since 0.08533s ago] ()


INFO:sqlalchemy.engine.Engine:[cached since 0.08533s ago] ()


User(id=1, name='spongebob', fullname='S. Squarepants')
2025-10-20 09:28:17,124 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK
